In [28]:
import time 

In [29]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

prompt_text = template.render(agent_type="Researcher-Agent", role="system")
print(prompt_text)

In [30]:
simulated_database = {
                "mediterranean diet": """The Mediterranean diet is rich in fruits,
                vegetables, whole grains, olive oil, and fish. Studies show it is associated with
                a lower risk of heart disease, improved brain health, and a longer lifespan. Key
                components include monounsaturated fats and antioxidants.""" }



In [69]:
from typing import Union
from pydantic import BaseModel , field_validator , Field 
from typing import Dict, Any , Union , TypedDict
import uuid


class ValidatorContext(BaseModel):
    task: str
    source_summary: str
    draft_post: str



class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: Any
    metadata: Dict[str, Any] = Field(default_factory=dict)


def create_mcp_message(sender: str ,  content: Any, metadata=None) -> MCPMessage:
    """Create a message in MCP format."""

    
    if not isinstance(content, (str, dict, ValidatorContext)):
        raise TypeError(
            f"content must be str, dict, or ValidatorContext; got {type(content).__name__}"
        )

        

    metadata = metadata or {} 
    metadata.setdefault("task_id", str(uuid.uuid4()))
    metadata.setdefault("parents",[])

    return MCPMessage(
        sender=sender,
        content=content,
        metadata=metadata   
    )

mcp_anser = create_mcp_message("agent_1", "Hello, this is a test message.", {"priority": "high" , "parents": ["writer-agent"]})
print(mcp_anser.sender)
print(mcp_anser.content)
print(mcp_anser.metadata)

agent_1
Hello, this is a test message.
{'priority': 'high', 'parents': ['writer-agent'], 'task_id': 'b00cfcf1-1164-40e1-a58f-485c6af6d071'}


### robust llm , retries and fallback

In [70]:
a = ValidatorContext(task = "this that" , source_summary= "thisthis" , draft_post= "wow")
create_mcp_message(sender= "fad" , content= a)

MCPMessage(protocol_version='1.0', sender='fad', content=ValidatorContext(task='this that', source_summary='thisthis', draft_post='wow'), metadata={'task_id': '3e209140-ced4-4a36-b4be-01932ef95f1b', 'parents': []})

In [55]:
from llm_clients import  google_langchain_client  , mimo_v2_flash_client
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage , SystemMessage
from settings import config

In [56]:
from loguru import logger 
from tenacity import RetryError, retry, stop_after_attempt, wait_exponential, retry_if_exception_type

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type(Exception),
    reraise=False
)
def _call_primary(message: list[BaseMessage]) -> AIMessage:
    return google_langchain_client.invoke(message)



def call_llm_robust(system_prompt:str, user_content:str):

    """ calling llm with retries and faallback """ 
    system_prompt= SystemMessage(content=system_prompt)
    user_prompt= HumanMessage(content=user_content)
    message = [system_prompt, user_prompt]

    try:
        logger.info("Invoking primary LLM.")
        return _call_primary(message).content

    ##################### fallback logic #####################
    except Exception as e:
        logger.warning(f"Primary LLM failed after retries: {e}. Switching to fallback.")
    try:
        return mimo_v2_flash_client.invoke(message).content ## fall back
    except Exception:
        return AIMessage(content="system error").content


## researcher agent 



In [57]:
mcp_anser.sender

'agent_1'

In [58]:
def researcher_agent(mcp_input: MCPMessage)-> MCPMessage:
    """ Takes a research topic , finds information and returns summary. """ 

    research_topic= mcp_input.content
    research_result = simulated_database.get(research_topic.lower(), "No Data found")
    
    system_prompt= template.render(agent_type="researcher_agent", role="system")
    summary= call_llm_robust(system_prompt, research_result)
    
    task_id= mcp_input.metadata.get("task_id")
    new_metadata= mcp_input.metadata


    return create_mcp_message(
        sender="researcher_agent",
        content=summary,
        metadata= {"task_id": task_id, "parents":[mcp_input.sender]})

## writer agent 

In [59]:
class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: str =Field(min_length=3)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [60]:
def writer_agent(mcp_input:MCPMessage) -> MCPMessage:
    """ This agent takes research findings and writes a shot blog post """ 
    
    research_summary= mcp_input.content
    system_prompt= template.render(agent_type="writer_agent", role="system")
    blog_post= call_llm_robust(system_prompt, research_summary)

    return create_mcp_message(
        sender="writer_agent",
        content=blog_post,
        metadata={"word count": len(blog_post.split()),"task_id": mcp_input.metadata.get("task_id"), "parents":[mcp_input.sender]})

        



In [61]:
mcp_for_researcher= create_mcp_message(
    sender="User",
    content="Mediterranean Diet",)

answer= researcher_agent(mcp_for_researcher)

2026-01-20 19:41:47.908 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [62]:
final_answer= writer_agent(answer)

2026-01-20 19:41:58.161 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [63]:
final_answer

MCPMessage(protocol_version='1.0', sender='writer_agent', content='## Unlock a Longer, Healthier Life: Embrace the Mediterranean Way!\n\nDreaming of a diet that’s as delicious as it is beneficial? Look no further than the vibrant Mediterranean way of eating! This time-tested eating pattern is bursting with flavor and packed with goodness.\n\nThink fresh fruits and veggies, hearty whole grains, and the liquid gold of olive oil. Add in plenty of fish for lean protein and those amazing omega-3s, and you’ve got a recipe for success. Studies show this lifestyle can significantly slash your risk of heart disease, boost your brainpower, and even contribute to a longer lifespan.\n\nReady to feel your best? Start incorporating these simple, wholesome foods into your meals today and savor the journey to a healthier you!', metadata={'word count': 122, 'task_id': '009501f8-f813-4f48-b9c5-7c650b609a79', 'parents': ['researcher_agent']})

## Validator Agent

In [76]:
def validator_agent(mcp_input:MCPMessage) -> MCPMessage:
    """ This agent validates the final blog post for quality and coherence """ 

    """ the mcp  inputs is like this sender = sender name ,
     contenxt = {
     "task" : "abcd" ,
     "summary" : "fadsfdasfda"
      "draft_post" : "dfafdada
      }""" 
    ctx = mcp_input.content
    if not isinstance(ctx ,ValidatorContext ):
        return create_mcp_message("validator_agent", "Error : Expected ValidatorContext , got string"   )


    system_prompt= template.render(agent_type="validator_agent", role="system")
    user= template.render(agent_type="validator_agent", role="user", **ctx.model_dump()) # This provides {{ task }}, {{ source_summary }}, and {{ draft_post }} to Jinja

    validation_report= call_llm_robust(system_prompt.strip(), user.strip() )

    return create_mcp_message(
        sender="validator_agent",
        content=validation_report,
        metadata={"task_id": mcp_input.metadata.get("task_id"), "parents":[mcp_input.sender]})

## final Orchestrator

## needs

In [77]:
simulated_database = {
                "mediterranean diet": """The Mediterranean diet is rich in fruits,
                vegetables, whole grains, olive oil, and fish. Studies show it is associated with
                a lower risk of heart disease, improved brain health, and a longer lifespan. Key
                components include monounsaturated fats and antioxidants.""" 
}


from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

In [80]:
def final_orchestrator(initial_goal):
    print("="*50, flush = True)
    print(f"Orchestrator started with goal: {initial_goal}")  
    print("="*50, flush = True)


    research_topic= "mediterranean diet"

    mcp_to_researcher= create_mcp_message(
        sender= "Orchestrator",
        content= research_topic,
    )

    #----------- research agent ----------- 
    mcp_from_researcher= researcher_agent(mcp_to_researcher)

    research_summary= mcp_from_researcher.content


    # ----------------- writer agent ----------------
    mcp_to_writer= create_mcp_message(
        sender="Orchestrator",
        content= mcp_from_researcher.content,)

    
    mcp_from_writer= writer_agent(mcp_to_writer)

    draft_post = mcp_from_writer.content
    #----------- create context -----------
    validation_contex= ValidatorContext(task = research_topic ,
                        source_summary= research_summary,
                        draft_post=draft_post) 

    # ---------- validation agent ---------------
    mcp_to_validator= create_mcp_message(
        sender="Orchestrator",
        content=validation_contex)

    mcp_from_validator= validator_agent(mcp_to_validator)
    validation_result= mcp_from_validator.content

    if "pass" in  validation_result.lower():
        return validation_result, mcp_from_validator, validation_contex
        

    else : 
        return  validation_result

In [81]:
user_goal = "Create a blog post about the benefits of the Mediterranean diet."

In [82]:
validation_result,mcp_from_validator, context_mcp = final_orchestrator(user_goal)

Orchestrator started with goal: Create a blog post about the benefits of the Mediterranean diet.


2026-01-20 19:47:16.674 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.
2026-01-20 19:47:23.418 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.
2026-01-20 19:47:25.116 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [83]:
validation_result

'pass'

In [87]:
context_mcp.task

'mediterranean diet'

In [93]:
print(context_mcp.source_summary)

The provided text offers a concise overview of the Mediterranean diet, highlighting its key components and associated health benefits. Here's an analysis based on that information and drawing upon broader research:

**Summary of Insights:**

*   **Dietary Composition:** The Mediterranean diet is characterized by a high intake of plant-based foods (fruits, vegetables, whole grains) and healthy fats (olive oil, fish).
*   **Health Benefits:** It is strongly linked to positive health outcomes, including:
    *   Reduced risk of cardiovascular disease.
    *   Enhanced brain health.
    *   Increased longevity.
*   **Key Nutrients:** Monounsaturated fats (primarily from olive oil) and antioxidants (abundant in fruits, vegetables, and olive oil) are identified as crucial bioactive components contributing to these benefits.

**Critical Analysis and Gaps in Current Knowledge (Based on the provided text, but extrapolating to common research themes):**

While the summary is accurate, it simplif

In [92]:
print(context_mcp.draft_post)

## Unlock Your Health: The Power of the Mediterranean Diet!

Dreaming of a healthier, longer life? Look no further than the vibrant Mediterranean diet! This delicious eating pattern, inspired by the traditional cuisines of countries bordering the Mediterranean Sea, is a true powerhouse for your well-being.

At its heart, the Mediterranean diet emphasizes **fresh fruits, colorful vegetables, hearty whole grains, and the magic of healthy fats** like olive oil and fatty fish. It's packed with **powerful antioxidants and beneficial monounsaturated fats**, working together to protect your heart and boost your brain health.

Studies consistently show that embracing this lifestyle can significantly **reduce your risk of heart disease, sharpen your mind, and even contribute to a longer, more vibrant life.** Ready to taste the goodness and feel the difference? Your journey to a healthier you starts with a plate full of delicious, wholesome foods!


In [90]:
context_mcp.model_dump()

{'task': 'mediterranean diet',
 'source_summary': 'The provided text offers a concise overview of the Mediterranean diet, highlighting its key components and associated health benefits. Here\'s an analysis based on that information and drawing upon broader research:\n\n**Summary of Insights:**\n\n*   **Dietary Composition:** The Mediterranean diet is characterized by a high intake of plant-based foods (fruits, vegetables, whole grains) and healthy fats (olive oil, fish).\n*   **Health Benefits:** It is strongly linked to positive health outcomes, including:\n    *   Reduced risk of cardiovascular disease.\n    *   Enhanced brain health.\n    *   Increased longevity.\n*   **Key Nutrients:** Monounsaturated fats (primarily from olive oil) and antioxidants (abundant in fruits, vegetables, and olive oil) are identified as crucial bioactive components contributing to these benefits.\n\n**Critical Analysis and Gaps in Current Knowledge (Based on the provided text, but extrapolating to common